<center><h1>第十章 时序数据</h1></center>

In [1]:
import numpy as np
import pandas as pd

## 一、时序中的基本对象

时间序列的概念在日常生活中十分常见，但对于一个具体的时序事件而言，可以从多个时间对象的角度来描述。例如2020年9月7日周一早上8点整需要到教室上课，这个课会在当天早上10点结束，其中包含了哪些时间概念？

* 第一，会出现时间戳（Date times）的概念，即'2020-9-7 08:00:00'和'2020-9-7 10:00:00'这两个时间点分别代表了上课和下课的时刻，在`pandas`中称为`Timestamp`。同时，一系列的时间戳可以组成`DatetimeIndex`，而将它放到`Series`中后，`Series`的类型就变为了`datetime64[ns]`，如果有涉及时区则为`datetime64[ns, tz]`，其中tz是timezone的简写。

* 第二，会出现时间差（Time deltas）的概念，即上课需要的时间，两个`Timestamp`做差就得到了时间差，pandas中利用`Timedelta`来表示。类似的，一系列的时间差就组成了`TimedeltaIndex`， 而将它放到`Series`中后，`Series`的类型就变为了`timedelta64[ns]`。

* 第三，会出现时间段（Time spans）的概念，即在8点到10点这个区间都会持续地在上课，在`pandas`利用`Period`来表示。类似的，一系列的时间段就组成了`PeriodIndex`， 而将它放到`Series`中后，`Series`的类型就变为了`Period`。

* 第四，会出现日期偏置（Date offsets）的概念，假设你只知道9月的第一个周一早上8点要去上课，但不知道具体的日期，那么就需要一个类型来处理此类需求。再例如，想要知道2020年9月7日后的第30个工作日是哪一天，那么时间差就解决不了你的问题，从而`pandas`中的`DateOffset`就出现了。同时，`pandas`中没有为一列时间偏置专门设计存储类型，理由也很简单，因为需求比较奇怪，一般来说我们只需要对一批时间特征做一个统一的特殊日期偏置。

通过这个简单的例子，就能够容易地总结出官方文档中的这个[表格](https://pandas.pydata.org/docs/user_guide/timeseries.html#overview)：

|概念 |                          单元素类型              |    数组类型         |                pandas数据类型|
|:---------|:----------|:-----------|:------------|
|Date times           |           `Timestamp`       |       `DatetimeIndex`  |   `datetime64[ns]`|
|Time deltas          |           `Timedelta`        |      `TimedeltaIndex` |  `timedelta64[ns]`|
|Time spans            |          `Period`           |      `PeriodIndex`   |    `period[freq]`|
|Date offsets          |          `DateOffset`         |    `None`          |    `None`|

由于时间段对象`Period/PeriodIndex`的使用频率并不高，因此将不进行讲解，而只涉及时间戳序列、时间差序列和日期偏置的相关内容。

## 二、时间戳
### 1. Timestamp的构造与属性

单个时间戳的生成利用`pd.Timestamp`实现，一般而言的常见日期格式都能被成功地转换：


In [ ]:
ts = pd.Timestamp('2022/10/18')
ts

In [ ]:
ts = pd.Timestamp('2022-10-18 08:10:30')
ts

通过`year, month, day, hour, min, second`可以获取具体的数值：

In [ ]:
ts.year

In [ ]:
ts.month

In [ ]:
ts.day

In [ ]:
ts.hour

In [ ]:
ts.minute

In [ ]:
ts.second

在`pandas`中，时间戳的最小精度为纳秒`ns`，由于使用了64位存储，可以表示的时间范围大约可以如下计算：
$$\rm Time\,Range = \frac{2^{64}}{10^9\times 60\times 60\times 24\times 365} \approx 585 (Years)$$
通过`pd.Timestamp.max`和`pd.Timestamp.min`可以获取时间戳表示的范围，可以看到确实表示的区间年数大小正如上述计算结果：

10$^9$ ns = 1 s

In [ ]:
pd.Timestamp.max

In [ ]:
pd.Timestamp.min

In [ ]:
pd.Timestamp.max.year - pd.Timestamp.min.year

### 2. Datetime序列的生成

一组时间戳可以组成时间序列，可以用`to_datetime`和`date_range`来生成。其中，`to_datetime`能够把一列时间戳格式的对象转换成为`datetime64[ns]`类型的时间序列：

In [ ]:
pd.to_datetime(['2020-1-1', '2020-1-3', '2020-1-6'])

In [ ]:
df = pd.read_csv('../data/learn_pandas.csv')
df

In [ ]:
df['Grade'].values

In [ ]:
s = pd.to_datetime(df.Test_Date)
s.head()

在极少数情况，时间戳的格式不满足转换时，可以强制使用`format`进行匹配：

In [ ]:
temp = pd.to_datetime(['2020\\1\\1','2020\\1\\3'],format='%Y\\%m\\%d')
temp

In [ ]:
temp = pd.to_datetime(['2020\\1\\1','2020\\1\\3'])
temp

注意上面由于传入的是列表，而非`pandas`内部的`Series`，因此返回的是`DatetimeIndex`，如果想要转为`datetime64[ns]`的序列，需要显式用`Series`转化：

In [ ]:
pd.Series(temp).head()

另外，还存在一种把表的多列时间属性拼接转为时间序列的`to_datetime`操作，此时的列名必须和以下给定的时间关键词列名一致：

In [ ]:
df_date_cols = pd.DataFrame({'year': [2020, 2020],
                             'month': [1, 1],
                             'day': [1, 2],
                             'hour': [10, 20],
                             'minute': [30, 50],
                             'second': [20, 40]})
pd.to_datetime(df_date_cols)

In [ ]:
df_date_cols = pd.DataFrame({'year': [2020, 2020],
                             'month': [1, 1],
                             'hour': [10, 20],
                             'minute': [30, 50],
                             'second': [20, 40]})
pd.to_datetime(df_date_cols)
# 年 月 日 是必不可少的三条数据

In [ ]:
df_date_cols = pd.DataFrame({'year': [2020, 2020],
                             'month': [1, 1],
                             'day': [1, 2],
                             'hour': [10, 20],
                             'second': [20, 40]})
pd.to_datetime(df_date_cols)

`date_range`是一种生成连续间隔时间的一种方法，其重要的参数为`start, end, freq, periods`，它们分别表示开始时间，结束时间，时间间隔，时间戳个数。其中，四个中的三个参数决定了，那么剩下的一个就随之确定了。这里要注意，开始或结束日期如果作为端点则它会被包含：

In [ ]:
pd.date_range('2020-1-1','2020-1-21', freq='10D') # 包含

In [ ]:
pd.date_range('2020-1-1','2020-2-28', freq='10D')

In [ ]:
pd.date_range('2020-1-1', '2020-2-28', periods=6) # 由于结束日期无法取到，freq不为10天

In [ ]:
pd.date_range('2020-1-1', '2020-2-20', periods=6)

这里的`freq`参数与`DateOffset`对象紧密相关，将在第四节介绍其具体的用法。
#### 【练一练】
`Timestamp`上定义了一个`value`属性，其返回的整数值代表了从1970年1月1日零点到给定时间戳相差的纳秒数，请利用这个属性构造一个随机生成给定日期区间内日期序列的函数。

In [ ]:
ts = pd.Timestamp('2022/10/18')
ts.value

In [ ]:
# 给定两个日期/时间节点，然后分别转换为纳秒数，
#再在这个区间内取一个随机数，
#然后再折算回日期。

def rand_ts(t1, t2):
    t1 = pd.to_datetime(t1)
    t2 = pd.to_datetime(t2)
    m1 = t1.value
    m2 = t2.value
    trand =np.random.randint(m1, m2, dtype='int64')
    return pd.to_datetime(trand)

In [ ]:
import datetime
datetime.datetime.now()

In [ ]:
t1 = '2022-6-26'
#t2 = '2022-8-30'
t2 = datetime.datetime.now()
rand_ts(t1, t2)

#### 【END】
最后，要介绍一种改变序列采样频率的方法`asfreq`，它能够根据给定的`freq`对序列进行类似于`reindex`的操作：

In [ ]:
s = pd.Series(np.random.rand(5), index=pd.to_datetime(['2020-1-%d'%i for i in range(1,10,2)]))
s.head()

In [ ]:
s.asfreq('D').head()

In [ ]:
s.asfreq('12H').head()

#### 【NOTE】
前面提到了`datetime64[ns]`本质上可以理解为一个大整数，对于一个该类型的序列，可以使用`max, min, mean`，来取得最大时间戳、最小时间戳和“平均”时间戳。
#### 【END】
### 3. dt对象

如同`category, string`的序列上定义了`cat, str`来完成分类数据和文本数据的操作，在时序类型的序列上定义了`dt`对象来完成许多时间序列的相关操作。这里对于`datetime64[ns]`类型而言，可以大致分为三类操作：取出时间相关的属性、判断时间戳是否满足条件、取整操作。

第一类操作的常用属性包括：`date, time, year, month, day, hour, minute, second, microsecond, nanosecond, dayofweek, dayofyear, weekofyear, daysinmonth, quarter`，其中`daysinmonth, quarter`分别表示该月一共有几天和季度。

In [ ]:
s = pd.Series(pd.date_range('2020-1-1','2020-1-3', freq='D'))
s.dt.date

In [ ]:
s.values

In [ ]:
s.dt.time

In [ ]:
s.dt.day

In [ ]:
s.dt.daysinmonth

In [ ]:
s.dt

在这些属性中，经常使用的是`dayofweek`，它返回了周中的星期情况，周一为0、周二为1，以此类推：

In [ ]:
s.dt.dayofweek

此外，可以通过`month_name, day_name`返回英文的月名和星期名，注意它们是方法而不是属性：

In [ ]:
s.dt.month_name()

In [ ]:
s.dt.day_name()

第二类判断操作主要用于测试是否为月/季/年的第一天或者最后一天：

In [ ]:
s.dt.is_year_start # 还可选 is_quarter/month_start

In [ ]:
s.dt.is_quarter_start

In [ ]:
s.dt.is_month_end

In [ ]:
s.dt.is_year_end # 还可选 is_quarter/month_end

第三类的取整操作包含`round, ceil, floor`，它们的公共参数为`freq`，常用的包括`H, min, S`（小时、分钟、秒），所有可选的`freq`可参考[此处](https://pandas.pydata.org/docs/user_guide/timeseries.html#offset-aliases>)。

In [ ]:
s = pd.Series(pd.date_range('2020-1-1 20:35:00', '2020-1-1 22:35:00', freq='45min'))
s

In [ ]:
s.dt.round('1H')

In [ ]:
s.dt.ceil('1H')

In [ ]:
s.dt.floor('1H')

### 4. 时间戳的切片与索引

一般而言，时间戳序列作为索引使用。如果想要选出某个子时间戳序列，第一类方法是利用`dt`对象和布尔条件联合使用，另一种方式是利用切片，后者常用于连续时间戳。下面，举一些例子说明：

In [ ]:
s = pd.Series(np.random.randint(2,size=366), index=pd.date_range('2020-01-01','2020-12-31'))
s

In [ ]:
idx = pd.Series(s.index).dt
idx

In [ ]:
pd.Series(s.index)

In [ ]:
s.head()

Example1：每月的第一天或者最后一天

In [ ]:
s[(idx.is_month_start|idx.is_month_end).values].head()

In [ ]:
s[(idx.is_month_start|idx.is_month_end)]

Example2：双休日

In [ ]:
s[idx.dayofweek.isin([5,6]).values].head()

Example3：取出单日值

In [ ]:
s['2020-01-01']

In [ ]:
s['20200101'] # 自动转换标准格式

Example4：取出七月

In [ ]:
s['2020-07'].head()

Example5：取出5月初至7月15日

In [ ]:
s['2020-05':'2020-7-15'].head()

In [ ]:
s['2020-05':'2020-7-15'].tail()

## 三、时间差
### 1. Timedelta的生成

正如在第一节中所说，时间差可以理解为两个时间戳的差，这里也可以通过`pd.Timedelta`来构造：

In [ ]:
pd.Timestamp('20200102 08:00:00')-pd.Timestamp('20200101 07:35:00')

In [ ]:
pd.Timedelta(days=1, minutes=25) # 需要注意加s

In [ ]:
pd.Timedelta('1 days 25 minutes') # 字符串生成

生成时间差序列的主要方式是`pd.to_timedelta`，其类型为`timedelta64[ns]`：

In [ ]:
df

In [ ]:
df.Time_Record

In [ ]:
s = pd.to_timedelta(df.Time_Record)

In [ ]:
s = pd.to_timedelta(df.Time_Record)
s

与`date_range`一样，时间差序列也可以用`timedelta_range`来生成，它们两者具有一致的参数：

In [ ]:
pd.timedelta_range('0s', '1000s', freq='6min')

In [ ]:
pd.timedelta_range('0s', '1000s', periods=3)

对于`Timedelta`序列，同样也定义了`dt`对象，上面主要定义了的属性包括`days, seconds, mircroseconds, nanoseconds`，它们分别返回了对应的时间差特征。需要注意的是，这里的`seconds`不是指单纯的秒，而是对天数取余后剩余的秒数：

In [ ]:
s.dt.seconds.head()

In [ ]:
s.dt.microseconds

In [ ]:
s.dt.days

如果不想对天数取余而直接对应秒数，可以使用`total_seconds`

In [ ]:
s.dt.total_seconds().head()

与时间戳序列类似，取整函数也是可以在`dt`对象上使用的：

In [ ]:
pd.to_timedelta(df.Time_Record).dt.round('min').head()
# 这里的 min 是按分钟

### 2. Timedelta的运算

时间差支持的常用运算有三类：与标量的乘法运算、与时间戳的加减法运算、与时间差的加减法与除法运算：

In [ ]:
td1 = pd.Timedelta(days=1)
td2 = pd.Timedelta(days=3)
ts = pd.Timestamp('20200101')
td1 * 2

In [ ]:
td2 - td1

In [ ]:
ts + td1

In [ ]:
ts - td1

这些运算都可以移植到时间差的序列上：

In [ ]:
td1 = pd.timedelta_range(start='1 days', periods=5)
td2 = pd.timedelta_range(start='12 hours', freq='2H', periods=5)
ts = pd.date_range('20200101', '20200105')
td1 * 5

In [ ]:
td1

In [ ]:
td2

In [ ]:
td1 * pd.Series(list(range(5))) # 逐个相乘

In [ ]:
td1 - td2

In [ ]:
td2 - td1
# 时间差 允许有负值出现

In [ ]:
td1 + pd.Timestamp('20200101')

In [ ]:
td1 + ts # 逐个相加

In [ ]:
td1

In [ ]:
ts

In [ ]:
td1[:-1] + ts
# 两个序列长度不同时会报错

## 四、日期偏置
### 1. Offset对象

日期偏置是一种和日历相关的特殊时间差，例如回到第一节中的两个问题：如何求2020年9月第一个周一的日期，以及如何求2020年9月7日后的第30个工作日是哪一天。

In [ ]:
pd.Timestamp('20200831') + pd.offsets.WeekOfMonth(week=0,weekday=0)

In [ ]:
pd.Timestamp('20200907') + pd.offsets.BDay(30)

从上面的例子中可以看到，`Offset`对象在`pd.offsets`中被定义。当使用`+`时获取离其最近的下一个日期，当使用`-`时获取离其最近的上一个日期：

In [ ]:
pd.Timestamp('20200831') - pd.offsets.WeekOfMonth(week=0,weekday=0)

In [ ]:
pd.Timestamp('20200907') - pd.offsets.BDay(30)

In [ ]:
pd.Timestamp('20200907') + pd.offsets.MonthEnd()

常用的日期偏置如下可以查阅这里的[文档](https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects)描述。在文档罗列的`Offset`中，需要介绍一个特殊的`Offset`对象`CDay`，其中的`holidays, weekmask`参数能够分别对自定义的日期和星期进行过滤，前者传入了需要过滤的日期列表，后者传入的是三个字母的星期缩写构成的星期字符串，其作用是只保留字符串中出现的星期：

In [ ]:
# holidays 对应的日期会被过滤掉，排除掉
my_filter = pd.offsets.CDay(n=1,weekmask='Wed Fri',holidays=['20200109'])
dr = pd.date_range('20200108', '20200111')
dr.to_series().dt.dayofweek

In [ ]:
[i + my_filter for i in dr]

In [ ]:
pd.to_datetime(['2020-01-15']).to_series().dt.dayofweek
# 说明是周三

In [ ]:
pd.to_datetime('2020-01-15')

上面的例子中，`n`表示增加一天`CDay`，`dr`中的第一天为`20200108`，但由于下一天`20200109`被排除了，并且`20200110`是合法的周五，因此转为`20200110`，其他后面的日期处理类似。
#### 【CAUTION】不要使用部分`Offset`
在当前版本下由于一些 ``bug`` ，不要使用 ``Day`` 级别以下的 ``Offset`` 对象，比如 ``Hour, Second`` 等，请使用对应的 ``Timedelta`` 对象来代替。
#### 【END】
### 2. 偏置字符串

前面提到了关于`date_range`的`freq`取值可用`Offset`对象，同时在`pandas`中几乎每一个`Offset`对象绑定了日期偏置字符串（`frequencies strings/offset aliases`），可以指定`Offset`对应的字符串来替代使用。下面举一些常见的例子。

In [ ]:
pd.date_range('20200101','20200331', freq='MS') # 月初 month begin

In [ ]:
pd.date_range('20200101','20200331', freq='M') # 月末

In [ ]:
pd.date_range('20200101','20200110', freq='B') # 工作日

In [ ]:
pd.date_range('20200101','20200201', freq='W-MON') # 周一

In [ ]:
pd.date_range('20200101','20200201', freq='WOM-1MON') # 每月第一个周一

In [ ]:
pd.date_range('20200101','20200201', freq='WOM-2TUE') # 每月第二个周二

上面的这些字符串，等价于使用如下的`Offset`对象：

In [ ]:
pd.date_range('20200101','20200331', freq=pd.offsets.MonthBegin())

In [ ]:
pd.date_range('20200101','20200331', freq=pd.offsets.MonthEnd())

In [ ]:
pd.date_range('20200101','20200110', freq=pd.offsets.BDay())

In [ ]:
pd.date_range('20200101','20200201', freq=pd.offsets.CDay(weekmask='Mon'))
# 这里的weekmask，得到的是认可的工作日

In [ ]:
pd.offsets.CDay(weekmask='Mon').weekmask

In [ ]:
pd.date_range('20200101','20200201', freq=pd.offsets.WeekOfMonth(week=0,weekday=0))

#### 【CAUTION】关于时区问题的说明
各类时间对象的开发，除了使用`python`内置的`datetime`模块，`pandas`还利用了`dateutil`模块，很大一部分是为了处理时区问题。总所周知，我国是没有夏令时调整时间一说的，但有些国家会有这种做法，导致了相对而言一天里可能会有23/24/25个小时，也就是`relativedelta`，这使得`Offset`对象和`Timedelta`对象有了对同一问题处理产生不同结果的现象，其中的规则也较为复杂，官方文档的写法存在部分描述错误，并且难以对描述做出统一修正，因为牵涉到了`Offset`相关的很多组件。因此，本教程完全不考虑时区处理，如果对时区处理的时间偏置有兴趣了解讨论，可以联系我或者参见[这里](https://github.com/pandas-dev/pandas/pull/36516)的讨论。
#### 【END】
## 五、时序中的滑窗与分组
### 1. 滑动窗口

所谓时序的滑窗函数，即把滑动窗口用`freq`关键词代替，下面给出一个具体的应用案例：在股票市场中有一个指标为`BOLL`指标，它由中轨线、上轨线、下轨线这三根线构成，具体的计算方法分别是`N`日均值线、`N`日均值加两倍`N`日标准差线、`N`日均值减两倍`N`日标准差线。利用`rolling`对象计算`N=30`的`BOLL`指标可以如下写出：

In [ ]:
import matplotlib.pyplot as plt
idx = pd.date_range('20200101', '20201231', freq='B')
np.random.seed(2020)
data = np.random.randint(-1,2,len(idx)).cumsum() # 随机游动构造模拟序列
s = pd.Series(data,index=idx)
s.head()

In [ ]:
data

In [ ]:
r = s.rolling('30D')
plt.plot(s)
plt.title('BOLL LINES')
plt.plot(r.mean())
plt.plot(r.mean()+r.std()*2)
plt.plot(r.mean()-r.std()*2)

对于`shift`函数而言，作用在`datetime64`为索引的序列上时，可以指定`freq`单位进行滑动：

In [ ]:
s.shift(freq='50D').head()

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html

shift，把数据进行偏移。

In [ ]:
s

In [ ]:
s.shift(freq='50D')
# 所有日期向后偏移了50天，

In [ ]:
s.rolling('30D').mean()

In [ ]:
r = s.shift(freq='50D')
plt.plot(s)
plt.title('BOLL LINES')
plt.plot(r.mean())
plt.plot(r.mean()+r.std()*2)
plt.plot(r.mean()-r.std()*2)

另外，`datetime64[ns]`的序列进行`diff`后就能够得到`timedelta64[ns]`的序列，这能够使用户方便地观察有序时间序列的间隔：

In [ ]:
my_series = pd.Series(s.index)
my_series.head()

In [ ]:
my_series.diff(1).head()

伯克希尔的数据


In [ ]:
bek = pd.read_excel('D:/xu/misc/berkshire-2021.xlsx', index_col='year')

In [ ]:
bk = pd.Series(bek['sccu-berkshire'][:-3], index=bek.index[:-3])

In [ ]:
#bk.index = pd.to_datetime(bk.index, format='%Y')

In [ ]:
r = bk.rolling(5)

In [ ]:
r = bk.rolling(5)
plt.plot(bk, color='r')
plt.title('BOLL LINES')
plt.plot(r.mean(), color='b')

In [ ]:
r = bk.rolling(5)
plt.plot(bk, color='r')
plt.title('BOLL LINES')
plt.plot(r.mean(), color='b')
plt.plot(r.mean()+r.std()*2, color='g')
plt.plot(r.mean()-r.std()*2)

### 2. 重采样

重采样对象`resample`和第四章中分组对象`groupby`的用法类似，前者是针对时间序列的分组计算而设计的分组对象。

例如，对上面的序列计算每10天的均值：

In [ ]:
s.resample('10D').mean().head()

In [ ]:
s.resample('10D').mean()

同时，如果没有内置定义的处理函数，可以通过`apply`方法自定义：

In [ ]:
s.resample('10D').apply(lambda x:x.max()-x.min()).head() # 极差

在`resample`中要特别注意组边界值的处理情况，默认情况下起始值的计算方法是从最小值时间戳对应日期的午夜`00:00:00`开始增加`freq`，直到不超过该最小时间戳的最大时间戳，由此对应的时间戳为起始值，然后每次累加`freq`参数作为分割结点进行分组，区间情况为左闭右开。下面构造一个不均匀的例子：

In [ ]:
idx = pd.date_range('20200101 8:26:35', '20200101 9:31:58', freq='77s')
data = np.random.randint(-1,2,len(idx)).cumsum()
s = pd.Series(data,index=idx)
s.head()

In [ ]:
s

下面对应的第一个组起始值为`08:24:00`，其是从当天0点增加72个`freq=7 min`得到的，如果再增加一个`freq`则超出了序列的最小时间戳`08:26:35`：

In [ ]:
s.resample('7min').mean().head()

有时候，用户希望从序列的最小时间戳开始依次增加`freq`进行分组，此时可以指定`origin`参数为`start`：

In [ ]:
s.resample('7min', origin='start').mean().head()

在返回值中，要注意索引一般是取组的第一个时间戳，但`M, A, Q, BM, BA, BQ, W`这七个是取对应区间的最后一个时间戳：

In [ ]:
s = pd.Series(np.random.randint(2,size=366), index=pd.date_range('2020-01-01', '2020-12-31'))
s.resample('M').mean().head()

In [ ]:
s.resample('MS').mean().head() # 结果一样，但索引不同

## 六、练习
### Ex1：太阳辐射数据集

现有一份关于太阳辐射的数据集：

In [ ]:
pd.read_csv('../data/solar.csv')

In [ ]:
df = pd.read_csv('../data/solar.csv', usecols=['Data','Time','Radiation','Temperature'])
df.head(3)

1. 将`Datetime, Time`合并为一个时间列`Datetime`，同时把它作为索引后排序。
2. 每条记录时间的间隔显然并不一致，请解决如下问题：
* 找出间隔时间的前三个最大值所对应的三组时间戳。
* 是否存在一个大致的范围，使得绝大多数的间隔时间都落在这个区间中？如果存在，请对此范围内的样本间隔秒数画出柱状图，设置`bins=50`。
3. 求如下指标对应的`Series`：
* 温度与辐射量的6小时滑动相关系数
* 以三点、九点、十五点、二十一点为分割，该观测所在时间区间的温度均值序列
* 每个观测6小时前的辐射量（一般而言不会恰好取到，此时取最近时间戳对应的辐射量）


1. 将`Datetime, Time`合并为一个时间列`Datetime`，同时把它作为索引后排序。

In [ ]:
df1 = df.copy()

In [ ]:
df1['Time']

In [ ]:
date = df1['Data'].str.extract(r'(\d{1,2}\/\d{1,2}\/\d{4})')
time = df1['Time']

In [ ]:
date = pd.Series(date[0])

In [ ]:
df1['Datetime'] = date.str.cat(time, sep=' ')

In [ ]:
df1['Datetime'] = pd.to_datetime(df1['Datetime'])

In [ ]:
df1

In [ ]:
df1.set_index('Datetime', inplace=True, drop=False)

In [ ]:
df1.sort_index(inplace=True)

In [ ]:
df1

2. 每条记录时间的间隔显然并不一致，请解决如下问题：
* 找出间隔时间的前三个最大值所对应的三组时间戳。
* 是否存在一个大致的范围，使得绝大多数的间隔时间都落在这个区间中？如果存在，请对此范围内的样本间隔秒数画出柱状图，设置`bins=50`。

In [ ]:
pd.to_timedelta(df1.index[0] - df1.index[1])

In [ ]:
pd.Series(pd.to_timedelta([df1.index[i+1] - df1.index[i] for i in range(df1.shape[0] - 1)]), index=df1.index[1:])

In [ ]:
df1['tdelta'] = pd.Series(pd.to_timedelta([df1.index[i+1] - df1.index[i] for i in range(df1.shape[0] - 1)]), index=df1.index[1:])

In [ ]:
delta = df1.sort_values('tdelta', ascending=False).head(3)

In [ ]:
delta

In [ ]:
df1.index.to_series().reset_index(drop=True).diff().dt.total_seconds()

In [ ]:
# 参考答案
s = df1.index.to_series().reset_index(drop=True).diff().dt.total_seconds()

In [ ]:
max_3 = s.nlargest(3).index

In [ ]:
max_3

In [ ]:
max_3.union(max_3-1)

In [ ]:
df1.index[max_3.union(max_3-1)]

In [ ]:
df1['tdelta'].dt.total_seconds().values

In [ ]:
tmp = df1['tdelta'].dt.seconds.values

In [ ]:
pd.Series(tmp).nlargest(20)

In [ ]:
pd.Series(tmp).nsmallest(20)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(df1['tdelta'].dt.seconds.values)

In [ ]:
(s>s.quantile(0.99))|(s<s.quantile(0.01))

In [ ]:
res = s.mask((s>s.quantile(0.99))|(s<s.quantile(0.01)))
# where the cond is False, keep the original value.
# where true, replace with corresponding value from other, default is nan
_ = plt.hist(res, 50)

3. 求如下指标对应的`Series`：
* 温度与辐射量的6小时滑动相关系数
* 以三点、九点、十五点、二十一点为分割，该观测所在时间区间的温度均值序列
* 每个观测6小时前的辐射量（一般而言不会恰好取到，此时取最近时间戳对应的辐射量）

In [ ]:
temp = pd.Series(df1['Temperature'], index=df1.index)
radi = pd.Series(df1['Radiation'], index=df1.index)

In [ ]:
res_temp = temp.rolling('6H')
res_radi = radi.rolling('6H')

In [ ]:
res_temp.corr(df1['Radiation'])

In [ ]:
df1['Radiation'].rolling('6H').corr(df1['Temperature'])

In [ ]:
temp.resample('6H', origin='03:00:00').mean()

In [ ]:
radi.shift(freq='-6H')

In [ ]:
# 参考答案
my_dt = df1.index.shift(freq='-6H')

In [ ]:
my_dt[20]

In [ ]:
df1.index.get_loc(my_dt[20], method='nearest')

In [ ]:
int_loc = [df1.index.get_indexer(i, method='nearest') for i in my_dt]

In [ ]:
my_dt

In [2]:
df = pd.read_csv('../data/solar.csv', usecols=['Data','Time','Radiation','Temperature'])
solar_date = df.Data.str.extract('([/|\w]+\s).+')[0]
df['Data'] = pd.to_datetime(solar_date + df.Time)
df = df.drop(columns='Time').rename(columns={'Data':'Datetime'}).set_index('Datetime').sort_index()
df.head(3)

s = df.index.to_series().reset_index(drop=True).diff().dt.total_seconds()
max_3 = s.nlargest(3).index
df.index[max_3.union(max_3-1)]

DatetimeIndex(['2016-09-29 23:55:26', '2016-10-01 00:00:19',
               '2016-11-29 19:05:02', '2016-12-01 00:00:02',
               '2016-12-05 20:45:53', '2016-12-08 11:10:42'],
              dtype='datetime64[ns]', name='Datetime', freq=None)

In [3]:
my_dt = df.index.shift(freq='-6H')


In [25]:
int_loc = [df.index.get_indexer([i], method='nearest') for i in my_dt]

In [36]:
int_loc = np.array(int_loc).reshape(-1)

In [38]:
res = df.Radiation.iloc[int_loc]
res.tail(3)

Datetime
2016-12-31 17:45:02    9.33
2016-12-31 17:50:01    8.49
2016-12-31 17:55:02    5.84
Name: Radiation, dtype: float64

In [3]:
# work for version 1.4.2
my_dt = df.index.shift(freq='-6H')
int_loc = [df.index.get_indexer([i], method='nearest') for i in my_dt]
int_loc = np.array(int_loc).reshape(-1)
res = df.Radiation.iloc[int_loc]
res.tail(3)

Datetime
2016-12-31 17:45:02    9.33
2016-12-31 17:50:01    8.49
2016-12-31 17:55:02    5.84
Name: Radiation, dtype: float64

In [6]:
my_dt = df.index.shift(freq='-6H')
int_loc = [df.index.get_indexer([i], method='nearest') for i in my_dt]
int_loc = np.array(int_loc).reshape(-1)
res = df.Radiation.iloc[int_loc]
res.index = df.index
res.tail(3)

Datetime
2016-12-31 23:45:04    9.33
2016-12-31 23:50:03    8.49
2016-12-31 23:55:01    5.84
Name: Radiation, dtype: float64

In [8]:
my_dt[-3:-1]

DatetimeIndex(['2016-12-31 17:45:04', '2016-12-31 17:50:03'], dtype='datetime64[ns]', name='Datetime', freq=None)

In [10]:
target = pd.DataFrame(
    {
        "Time": df.index.shift(freq='-6H'),
        "Datetime": df.index,
    }
)
res = pd.merge_asof(
    target,
    df.reset_index().rename(columns={"Datetime": "Time"}),
    left_on="Time",
    right_on="Time",
    direction="nearest"
).set_index("Datetime").Radiation
res

Datetime
2016-09-01 00:00:08     2.58
2016-09-01 00:05:10     2.58
2016-09-01 00:20:06     2.58
2016-09-01 00:25:05     2.58
2016-09-01 00:30:09     2.58
                       ...  
2016-12-31 23:35:02    15.96
2016-12-31 23:40:01    11.98
2016-12-31 23:45:04     9.33
2016-12-31 23:50:03     8.49
2016-12-31 23:55:01     5.84
Name: Radiation, Length: 32686, dtype: float64

### Ex2：水果销量数据集

现有一份2019年每日水果销量记录表：

In [198]:
df = pd.read_csv('../data/fruit.csv')
df.head(3)

,Date,Fruit,Sale
0,2019-04-18,Peach,15
1,2019-12-29,Peach,15
2,2019-06-05,Peach,19


In [119]:
df1 = df.copy()

1. 统计如下指标：
* 每月上半月（15号及之前）与下半月葡萄销量的比值
* 每月最后一天的生梨销量总和
* 每月最后一天工作日的生梨销量总和
* 每月最后五天的苹果销量均值
2. 按月计算周一至周日各品种水果的平均记录条数，行索引外层为水果名称，内层为月份，列索引为星期。
3. 按天计算向前10个工作日窗口的苹果销量均值序列，非工作日的值用上一个工作日的结果填充。


1. 统计如下指标：
* 每月上半月（15号及之前）与下半月葡萄销量的比值
* 每月最后一天的生梨销量总和
* 每月最后一天工作日的生梨销量总和
* 每月最后五天的苹果销量均值

In [121]:
# * 每月上半月（15号及之前）与下半月葡萄销量的比值
df1['Date'] = pd.to_datetime(df1['Date']).values

In [128]:
df1.set_index('Date', drop=True, inplace=True)

In [133]:
#df1.set_index(df1['Date'], drop=True, inplace=True)
#如果写成这句，得到的表中，就会还有 Date 列，如果写上面那句，得到的表中，就没有Date列，而只有index

In [37]:
df1['Fruit'].value_counts()

Peach     5665
Grape     4368
Pear      3944
Apple     3657
Banana    2366
Name: Fruit, dtype: int64

In [44]:
# 每月上半月和下半月葡萄销量的比值
grape = df1[df1['Fruit']=='Grape'].sort_index()

In [57]:
grape['Sale'].resample('SM').mean()
# https://pandas.pydata.org/docs/user_guide/timeseries.html
# 每个半月的最后一天计算销量平均值

Date
2018-12-31    64.512048
2019-01-15    58.217617
2019-01-31    60.806061
2019-02-15    60.791667
2019-02-28    58.922619
2019-03-15    59.572072
2019-03-31    60.123596
2019-04-15    58.386243
2019-04-30    57.751445
2019-05-15    61.402174
2019-05-31    64.537143
2019-06-15    55.585366
2019-06-30    66.331429
2019-07-15    55.647799
2019-07-31    64.973958
2019-08-15    61.561224
2019-08-31    63.340314
2019-09-15    59.664894
2019-09-30    60.212121
2019-10-15    61.060000
2019-10-31    61.016129
2019-11-15    55.032895
2019-11-30    59.938650
2019-12-15    62.664835
Freq: SM-15, Name: Sale, dtype: float64

In [60]:
grape['Sale'].resample('SMS').mean()

Date
2019-01-01    64.512048
2019-01-15    58.907767
2019-02-01    60.092105
2019-02-15    60.624309
2019-03-01    58.961290
2019-03-15    59.230435
2019-04-01    60.611765
2019-04-15    59.150754
2019-05-01    56.779141
2019-05-15    61.360406
2019-06-01    64.839506
2019-06-15    56.250000
2019-07-01    65.940476
2019-07-15    55.791667
2019-08-01    65.300546
2019-08-15    61.656863
2019-09-01    63.311475
2019-09-15    59.356784
2019-10-01    60.572193
2019-10-15    62.018692
2019-11-01    59.819767
2019-11-15    55.918750
2019-12-01    59.277419
2019-12-15    62.664835
Freq: SMS-15, Name: Sale, dtype: float64

In [71]:
# 参考答案
res = grape.groupby([np.where(grape.Date.dt.day<=15, 'First', 'Second'), 
                     grape.Date.dt.month])['Sale'].mean().to_frame().unstack(0).droplevel(0, axis=1)

In [72]:
res

,First,Second
Date,,
1,66.349462,56.467742
2,59.447059,61.355828
3,57.502890,60.443396
4,60.437838,59.206522
5,57.135593,61.366120
6,64.923977,55.798030
7,65.653631,55.407643
8,64.651515,62.047619
9,63.297436,59.117647


In [58]:
res = (res.First / res.Second).rename_axis("Month")

In [59]:
res

Month
1     1.174998
2     0.968890
3     0.951351
4     1.020797
5     0.931061
6     1.163553
7     1.184920
8     1.041966
9     1.070703
10    1.005624
11    1.026276
12    0.972197
dtype: float64

In [68]:
tmp = grape[(grape.Date.dt.day<=15) & (grape.Date.dt.month==1)]

In [70]:
tmp['Sale'].mean()

66.34946236559139

In [74]:
# 每月最后一天的生梨销量总和
pear = df1[df1['Fruit']=='Pear'].sort_index()

In [77]:
pear['Sale'].resample('M').sum()

Date
2019-01-31    20756
2019-02-28    18594
2019-03-31    18459
2019-04-30    19021
2019-05-31    21578
2019-06-30    17888
2019-07-31    21544
2019-08-31    20872
2019-09-30    19801
2019-10-31    21238
2019-11-30    19037
2019-12-31    19565
Freq: M, Name: Sale, dtype: int64

In [98]:
# 答案
pear[pear.Date.dt.is_month_end].Sale.groupby('Date').sum()

Date
2019-01-31    847
2019-02-28    774
2019-03-31    761
2019-04-30    648
2019-05-31    616
2019-06-30    179
2019-07-31    757
2019-08-31    813
2019-09-30    858
2019-10-31    753
2019-11-30    859
Name: Sale, dtype: int64

In [100]:
pd.date_range('20190101', '20191231', freq='BM')

DatetimeIndex(['2019-01-31', '2019-02-28', '2019-03-29', '2019-04-30',
               '2019-05-31', '2019-06-28', '2019-07-31', '2019-08-30',
               '2019-09-30', '2019-10-31', '2019-11-29', '2019-12-31'],
              dtype='datetime64[ns]', freq='BM')

In [104]:
# 每月最后一天工作日的生梨销量总和
pear[pear.Date.isin(pd.date_range('20190101', '20191231', freq='BM'))]['Sale'].groupby('Date').mean()

Date
2019-01-31    60.500000
2019-02-28    59.538462
2019-03-29    56.666667
2019-04-30    64.800000
2019-05-31    61.600000
2019-06-28    67.222222
2019-07-31    63.083333
2019-08-30    71.714286
2019-09-30    78.000000
2019-10-31    62.750000
2019-11-29    62.789474
Name: Sale, dtype: float64

In [232]:
# 每月最后五天的苹果销量均值
apple = df1[df1['Fruit']=='Apple'].sort_index()

In [200]:
df.Date = pd.to_datetime(df.Date)
df.Date.drop_duplicates().dt.month

0        4
1       12
2        6
3       10
4        6
        ..
1479     1
1545    10
1621    10
1691     5
1716     7
Name: Date, Length: 364, dtype: int64

In [205]:
df.drop_duplicates().groupby(df.Date.drop_duplicates().dt.month)['Date'].nlargest(5).reset_index(drop=True)

0    2019-01-31
1    2019-01-30
2    2019-01-29
3    2019-01-28
4    2019-01-27
5    2019-02-28
6    2019-02-27
7    2019-02-26
8    2019-02-25
9    2019-02-24
10   2019-03-31
11   2019-03-30
12   2019-03-29
13   2019-03-28
14   2019-03-27
15   2019-04-30
16   2019-04-29
17   2019-04-28
18   2019-04-27
19   2019-04-26
20   2019-05-31
21   2019-05-30
22   2019-05-29
23   2019-05-28
24   2019-05-27
25   2019-06-30
26   2019-06-29
27   2019-06-28
28   2019-06-27
29   2019-06-26
30   2019-07-31
31   2019-07-30
32   2019-07-29
33   2019-07-28
34   2019-07-27
35   2019-08-31
36   2019-08-30
37   2019-08-29
38   2019-08-28
39   2019-08-27
40   2019-09-30
41   2019-09-29
42   2019-09-28
43   2019-09-27
44   2019-09-26
45   2019-10-31
46   2019-10-30
47   2019-10-29
48   2019-10-28
49   2019-10-27
50   2019-11-30
51   2019-11-29
52   2019-11-28
53   2019-11-27
54   2019-11-26
55   2019-12-30
56   2019-12-29
57   2019-12-28
58   2019-12-27
59   2019-12-26
Name: Date, dtype: datetime64[ns]

In [224]:
df.Date.drop_duplicates().dt.month

0        4
1       12
2        6
3       10
4        6
        ..
1479     1
1545    10
1621    10
1691     5
1716     7
Name: Date, Length: 364, dtype: int64

In [242]:
apple.reset_index(inplace=True)

In [245]:
target_dt = apple.drop_duplicates().groupby(apple.Date.drop_duplicates().dt.month)['Date'].nlargest(5).reset_index(drop=True)

In [251]:
res

,Date,Fruit,Sale
0,2019-01-31,Apple,65
1,2019-01-31,Apple,36
2,2019-01-31,Apple,59
3,2019-01-31,Apple,72
4,2019-01-31,Apple,74
...,...,...,...
583,2019-12-26,Apple,126
584,2019-12-26,Apple,34
585,2019-12-26,Apple,17
586,2019-12-26,Apple,78


In [259]:
res = apple.set_index('Date').loc[target_dt].reset_index()

In [260]:
res = res.groupby(res.Date.dt.month)['Sale'].mean().rename_axis('Month')

In [261]:
res

Month
1     65.313725
2     54.061538
3     59.325581
4     65.795455
5     57.465116
6     61.897436
7     57.000000
8     73.636364
9     62.301887
10    59.562500
11    64.437500
12    66.020000
Name: Sale, dtype: float64

2. 按月计算周一至周日各品种水果的平均记录条数，行索引外层为水果名称，内层为月份，列索引为星期。

In [262]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']

week_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sum']

In [264]:
group1 = df.Date.dt.month_name().astype('category').cat.reorder_categories(month_order, ordered=True)

In [265]:
group1

0            April
1         December
2             June
3          October
4             June
           ...    
19995         July
19996      October
19997        April
19998    September
19999    September
Name: Date, Length: 20000, dtype: category
Categories (12, object): ['January' < 'February' < 'March' < 'April' ... 'September' < 'October' < 'November' < 'December']

In [267]:
group2 = df.Fruit

In [270]:
group3 = df.Date.dt.dayofweek.replace(dict(zip(range(7), week_order))).astype('category').cat.reorder_categories(week_order, ordered=True)

In [271]:
group3

0        Thu
1        Sum
2        Wed
3        Mon
4        Tue
        ... 
19995    Sat
19996    Mon
19997    Thu
19998    Sum
19999    Fri
Name: Date, Length: 20000, dtype: category
Categories (7, object): ['Mon' < 'Tue' < 'Wed' < 'Thu' < 'Fri' < 'Sat' < 'Sum']

In [295]:
df.groupby([group1, group2, group3])['Sale'].count().to_frame().unstack(0).droplevel(0, axis=1)

Date         January  February  March  April  May  June  July  August  \
Fruit  Date                                                             
Apple  Mon        46        43     43     47   43    40    41      38   
       Tue        50        40     44     52   46    39    50      42   
       Wed        50        47     37     43   39    39    58      43   
       Thu        45        35     31     47   58    33    52      44   
       Fri        32        33     52     31   46    38    37      48   
       Sat        42        38     42     35   55    58    40      67   
       Sum        23        45     42     37   31    52    34      46   
Banana Mon        27        24     29     36   20    27    26      30   
       Tue        29        24     19     24   23    22    33      24   
       Wed        24        35     18     28   35    31    23      24   
       Thu        42        23     22     33   26    32    24      39   
       Fri        36        35     32     24   34    22    28      30   
       Sat        24        27     37     23   32    37    29      35   
       Sum        35        23     31     27   31    28    14      43   
Grape  Mon        42        53     52     61   52    42    50      41   
       Tue        75        39     62     71   48    53    55      42   
       Wed        53        42     39     34   53    56    52      53   
       Thu        63        46     51     49   71    58    37      72   
       Fri        36        62     64     51   55    46    49      72   
       Sat        57        44     60     54   40    60    50      45   
       Sum        46        47     57     49   41    59    43      62   
Peach  Mon        67        75     61     86   60    74    64      63   
       Tue        78        68     56     91   64    59    51      59   
       Wed        73        61     69     55   84    65    90      55   
       Thu        88        55     49     75   77    64    63      94   
       Fri        59        58     78     65   71    61    50      66   
       Sat        49        56     73     74   80    74    65      75   
       Sum        72        62     81     57   74    73    64      64   
Pear   Mon        39        44     42     57   45    28    57      45   
       Tue        69        42     43     48   40    45    50      47   
       Wed        51        49     36     47   60    46    46      31   
       Thu        54        46     41     36   59    40    38      63   
       Fri        48        55     49     45   58    43    50      43   
       Sat        36        35     51     47   45    62    41      50   
       Sum        40        54     58     39   45    45    63      44   

Date         September  October  November  December  
Fruit  Date                                          
Apple  Mon          59       42        39        45  
       Tue          40       57        47        47  
       Wed          35       46        47        38  
       Thu          36       63        37        40  
       Fri          34       37        46        41  
       Sat          45       35        53        47  
       Sum          49       51        37        55  
Banana Mon          30       27        19        32  
       Tue          17       29        26        28  
       Wed          34       38        20        21  
       Thu          24       34        22        27  
       Fri          23       27        30        28  
       Sat          31       34        32        30  
       Sum          34       21        27        28  
Grape  Mon          57       66        37        59  
       Tue          55       63        48        38  
       Wed          58       73        44        50  
       Thu          40       62        45        36  
       Fri          51       42        58        47  
       Sat          55       47        56        44  
       Sum          66       48        44        63  
Peach  Mon          82       62        54        81  
  

In [272]:
res = df.groupby([group1, group2, group3])['Sale'].count().to_frame().unstack(0).droplevel(0, axis=1)

In [276]:
res

Date         January  February  March  April  May  June  July  August  \
Fruit  Date                                                             
Apple  Mon        46        43     43     47   43    40    41      38   
       Tue        50        40     44     52   46    39    50      42   
       Wed        50        47     37     43   39    39    58      43   
       Thu        45        35     31     47   58    33    52      44   
       Fri        32        33     52     31   46    38    37      48   
       Sat        42        38     42     35   55    58    40      67   
       Sum        23        45     42     37   31    52    34      46   
Banana Mon        27        24     29     36   20    27    26      30   
       Tue        29        24     19     24   23    22    33      24   
       Wed        24        35     18     28   35    31    23      24   
       Thu        42        23     22     33   26    32    24      39   
       Fri        36        35     32     24   34    22    28      30   
       Sat        24        27     37     23   32    37    29      35   
       Sum        35        23     31     27   31    28    14      43   
Grape  Mon        42        53     52     61   52    42    50      41   
       Tue        75        39     62     71   48    53    55      42   
       Wed        53        42     39     34   53    56    52      53   
       Thu        63        46     51     49   71    58    37      72   
       Fri        36        62     64     51   55    46    49      72   
       Sat        57        44     60     54   40    60    50      45   
       Sum        46        47     57     49   41    59    43      62   
Peach  Mon        67        75     61     86   60    74    64      63   
       Tue        78        68     56     91   64    59    51      59   
       Wed        73        61     69     55   84    65    90      55   
       Thu        88        55     49     75   77    64    63      94   
       Fri        59        58     78     65   71    61    50      66   
       Sat        49        56     73     74   80    74    65      75   
       Sum        72        62     81     57   74    73    64      64   
Pear   Mon        39        44     42     57   45    28    57      45   
       Tue        69        42     43     48   40    45    50      47   
       Wed        51        49     36     47   60    46    46      31   
       Thu        54        46     41     36   59    40    38      63   
       Fri        48        55     49     45   58    43    50      43   
       Sat        36        35     51     47   45    62    41      50   
       Sum        40        54     58     39   45    45    63      44   

Date         September  October  November  December  
Fruit  Date                                          
Apple  Mon          59       42        39        45  
       Tue          40       57        47        47  
       Wed          35       46        47        38  
       Thu          36       63        37        40  
       Fri          34       37        46        41  
       Sat          45       35        53        47  
       Sum          49       51        37        55  
Banana Mon          30       27        19        32  
       Tue          17       29        26        28  
       Wed          34       38        20        21  
       Thu          24       34        22        27  
       Fri          23       27        30        28  
       Sat          31       34        32        30  
       Sum          34       21        27        28  
Grape  Mon          57       66        37        59  
       Tue          55       63        48        38  
       Wed          58       73        44        50  
       Thu          40       62        45        36  
       Fri          51       42        58        47  
       Sat          55       47        56        44  
       Sum          66       48        44        63  
Peach  Mon          82       62        54        81  
  

3. 按天计算向前10个工作日窗口的苹果销量均值序列，非工作日的值用上一个工作日的结果填充。

In [299]:
df_apple = df[(df.Fruit=='Apple')&(~df.Date.dt.dayofweek.isin([5, 6]))]

In [302]:
s = pd.Series(df_apple.Sale.values, index=df_apple.Date).groupby('Date').sum()
s

Date
2019-01-01    189
2019-01-02    482
2019-01-03    890
2019-01-04    550
2019-01-07    494
             ... 
2019-12-24    863
2019-12-25    919
2019-12-26    650
2019-12-27    808
2019-12-30    531
Length: 260, dtype: int64

In [303]:
res = s.rolling('10D').mean().reindex(pd.date_range('20190101', '20191231')).fillna(method='ffill')

In [304]:
res

2019-01-01    189.000000
2019-01-02    335.500000
2019-01-03    520.333333
2019-01-04    527.750000
2019-01-05    527.750000
                 ...    
2019-12-27    626.875000
2019-12-28    626.875000
2019-12-29    626.875000
2019-12-30    710.166667
2019-12-31    710.166667
Freq: D, Length: 365, dtype: float64